In [ ]:
# 1️⃣ تثبيت المكتبات
!pip install transformers torch googletrans==4.0.0-rc1 SpeechRecognition pydub



In [ ]:
# 2️⃣ استيراد المكتبات
import difflib
import os
from transformers import pipeline
from googletrans import Translator
import speech_recognition as sr
from pydub import AudioSegment
from google.colab import files
from IPython.display import display, Markdown


In [ ]:
# 3️⃣ تحميل الموديلات
display(Markdown("⏳ جاري تحميل موديل تصحيح الجرامر..."))
corrector = pipeline(
    "text2text-generation",
    model="vennify/t5-base-grammar-correction"
)

chatbot = pipeline("text-generation", model="microsoft/DialoGPT-medium")
translator = Translator()

⏳ جاري تحميل موديل تصحيح الجرامر...

Device set to use cpu
Device set to use cpu


In [ ]:
# 4️⃣ الدوال المساعدة

def correct_sentence(sentence, lang="en"):
    if lang.lower().startswith("en"):
        result = corrector(sentence, max_length=100, num_return_sequences=1)
        return result[0]['generated_text']
    else:
        return sentence

def find_wrong_words(original, corrected):
    diff = difflib.ndiff(original.split(), corrected.split())
    wrong_words = [word[2:] for word in diff if word.startswith("- ")]
    return wrong_words

def general_question(question, dest_lang="ar"):
    return translator.translate(question, dest=dest_lang).text

def audio_to_text(file_path, lang="en"):
    recognizer = sr.Recognizer()
    wav_path = file_path.rsplit('.',1)[0] + ".wav"
    try:
        sound = AudioSegment.from_file(file_path)
        sound.export(wav_path, format="wav")
    except:
        return "❌ خطأ في تحويل ملف الصوت"

    with sr.AudioFile(wav_path) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio, language=lang)
        return text
    except sr.UnknownValueError:
        return "❌ لم يتم التعرف على الكلام"
    except sr.RequestError:
        return "❌ مشكلة في الاتصال بخدمة Google"
    finally:
        if os.path.exists(wav_path):
            os.remove(wav_path)

def chat_mode():
    print("💬 دخلنا في المحادثة الحرة. اكتب رسالتك (للخروج اكتب exit).")
    history = ""
    while True:
        user_msg = input("👤 انت: ")
        if user_msg.lower() in ["exit", "خروج", "quit"]:
            print("👋 انتهاء المحادثة.")
            break

        # بناء السياق
        history += f"\nUser: {user_msg}\nBot:"

        # توليد الرد
        result = chatbot(
            history,
            max_new_tokens=150,   # استخدام max_new_tokens فقط
            pad_token_id=50256,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

        # استخراج الرد بدون تكرار
        generated_text = result[0]["generated_text"]
        reply = generated_text.split("Bot:")[-1].strip()

        print("🤖 البوت:", reply)

        # تحديث التاريخ
        history += f" {reply}"



In [ ]:
# 5️⃣ البوت الرئيسي
def language_practice_bot():
    display(Markdown("👋 **أهلاً! أنا البوت اللغوي متعدد اللغات.**"))
    display(Markdown("🟢 تقدر:"))
    display(Markdown("1️⃣ تكتب جملة للتصحيح"))
    display(Markdown("2️⃣ ترفع ملف صوت للتحويل + التصحيح"))
    display(Markdown("3️⃣ تسأل سؤال عام (معاني / ترجمة / قواعد)"))
    display(Markdown("4️⃣ تدخل في محادثة حرة مع البوت (أي موضوع)"))
    display(Markdown("لو عايز تخرج اكتب: `exit`"))

    while True:
        mode = input("\nاختر الوضع (1/2/3/4): ")

        if mode in ["خروج", "exit", "quit"]:
            display(Markdown("👋 **باي باي! استمر في التدريب 🎉**"))
            break

        elif mode == "1":
            lang = input("🌐 اختر لغة الإدخال (مثال en / ar / fr): ")
            user_input = input("✏️ اكتب جملتك: ")
            corrected = correct_sentence(user_input, lang)
            wrong = find_wrong_words(user_input, corrected)
            display(Markdown(f"✅ النص المصحح: {corrected}"))
            if wrong:
                display(Markdown(f"⚠️ كلمات محتاجة تعديل: {', '.join(wrong)}"))

        elif mode == "2":
            lang = input("🌐 اختر لغة الإدخال (مثال en / ar): ")
            uploaded = files.upload()
            filename = next(iter(uploaded))
            spoken_text = audio_to_text(filename, lang)
            print("📝 النص:", spoken_text)
            if "❌" not in spoken_text:
                corrected = correct_sentence(spoken_text, lang)
                wrong = find_wrong_words(spoken_text, corrected)
                display(Markdown(f"✅ النص بعد التصحيح: {corrected}"))
                if wrong:
                    display(Markdown(f"⚠️ كلمات اتقالت غلط: {', '.join(wrong)}"))
            os.remove(filename)

        elif mode == "3":
            q = input("❓ اكتب سؤالك: ")
            dest = input("🌐 اختر لغة الإجابة (افتراضي ar): ") or "ar"
            answer = general_question(q, dest)
            display(Markdown(f"📖 الإجابة: {answer}"))

        elif mode == "4":
            chat_mode()

        else:
            print("❌ اختيار غير صحيح.")




# 6️⃣ تشغيل البوت

language_practice_bot()

👋 **أهلاً! أنا البوت اللغوي متعدد اللغات.**

🟢 تقدر:

1️⃣ تكتب جملة للتصحيح

2️⃣ ترفع ملف صوت للتحويل + التصحيح

3️⃣ تسأل سؤال عام (معاني / ترجمة / قواعد)

...

📖 الإجابة: apple


اختر الوضع (1/2/3/4): 3
❓ اكتب سؤالك: apple
🌐 اختر لغة الإجابة (افتراضي ar): fr


📖 الإجابة: pomme


اختر الوضع (1/2/3/4): exit


👋 **باي باي! استمر في التدريب 🎉**